In [2]:
from agent import program
from agent import Agent
from referee.game import Board, SpawnAction, SpreadAction, HexPos, HexDir, PlayerColor, constants
my_board = Board()
action_1 = SpawnAction(HexPos(1,1))

my_board.apply_action(action_1)

# print(my_board.render())

In [2]:
# 提取所有棋子颜色的坐标 & 空坐标，并存在list里面
from typing import List, Optional

def extract_positions(board, color: Optional[PlayerColor] = None) -> List[HexPos]:
    positions = []

    for position, cell_state in board._state.items():
        if color is None and cell_state.player is None:  # Extract empty positions
            positions.append(position)
        elif cell_state.player == color:  # Extract colored positions
            positions.append(position)

    return positions

In [3]:
# 给每个棋子都设定6个方向的spread
def generate_spread_actions(color, board) -> List[SpreadAction]:
    colored_positions = extract_positions(board, color)
    spread_actions = []

    for position in colored_positions:
        for direction in HexDir:
            neighbor_pos = position + direction
            if board._within_bounds(neighbor_pos):
                spread_action = SpreadAction(cell=position, direction=direction)
                spread_actions.append(spread_action)

    return spread_actions

In [4]:
# SPAWN actions
def generate_spawn_actions(color, board) -> List[SpawnAction]:
    spawn_actions = []
    empty_list = extract_positions(board)

    for position in empty_list:
        if board._within_bounds(position):
            spawn_action = SpawnAction(cell=position)
            spawn_actions.append(spawn_action)

    return spawn_actions

In [5]:

from typing import Union
def get_legal_actions(color, board) -> List[Union[SpawnAction, SpreadAction]]:
    spawn_actions = generate_spawn_actions(color, board)
    spread_actions = generate_spread_actions(color, board)
    all_actions = spawn_actions + spread_actions

    red_power = board._color_power(PlayerColor.RED)
    blue_power = board._color_power(PlayerColor.BLUE)

    if(red_power+blue_power <= 49):
        return all_actions


In [6]:
def is_terminal(game_state):
    return game_state.game_over

1. Implement a game state evaluation function: Create a function that assigns a numerical value to a given game state, representing the desirability of the state for each agent. The evaluation function should consider the position of pieces, material balance, and other relevant factors specific to your game.

2. Define the utility function: Create a utility function to determine if a game state is a terminal state (i.e., the game has ended) and return the corresponding utility value.

3. Implement the minimax algorithm with alpha-beta pruning: Write a recursive function that explores the game tree up to a certain depth. The function alternates between maximizing and minimizing layers, corresponding to the Red agent's turn and the Blue agent's turn, respectively. Use alpha-beta pruning to improve efficiency by pruning branches of the game tree that do not need to be explored.

In [7]:
red_list = extract_positions(my_board, PlayerColor.RED)
# red_list
blue_list = extract_positions(my_board, PlayerColor.BLUE)
# blue_list

In [8]:
# def evaluate_state_and_complexity(game_state: Board):
#     # Calculate the total power for each player
#     red_power = game_state._color_power(PlayerColor.RED)
#     blue_power = game_state._color_power(PlayerColor.BLUE)

#     # Calculate the power difference between players
#     power_difference = red_power - blue_power

#     # Calculate the number of pieces for each player
#     red_pieces = len(game_state._player_cells(PlayerColor.RED))
#     blue_pieces = len(game_state._player_cells(PlayerColor.BLUE))

#     # Calculate the piece difference between players
#     piece_difference = red_pieces - blue_pieces

#     empty_neighbors_total = 0
#     empty_count = 0
#     for pos, cell in game_state._state.items():
#         if cell.player is not None:
#             piece_count[cell.player] += 1
#             power_difference[cell.player] += cell.power
#             empty_neighbor_count[cell.player] += game_state.empty_neighbors(pos)
    
#     # Assign weights to the factors and compute the final score
#     power_weight = 0.5
#     piece_weight = 1.0
#     empty_neighbor_weight = 0.25
#     score = (power_weight * power_difference +
#              piece_weight * piece_difference +
#              empty_neighbor_weight * empty_neighbors_total)

#     # Estimate complexity
#     total_power = red_power + blue_power
#     total_pieces = red_pieces + blue_pieces
#     complexity = total_power * 0.5 + total_pieces * 0.5

#     return score, complexity
def evaluate_state_and_complexity(game_state: Board):
    # Calculate the total power for each player
    red_power = game_state._color_power(PlayerColor.RED)
    blue_power = game_state._color_power(PlayerColor.BLUE)

    # Calculate the power difference between players
    power_difference = red_power - blue_power

    # Calculate the number of pieces for each player
    # red_cells = game_state._player_cells(PlayerColor.RED)
    # blue_cells = game_state._player_cells(PlayerColor.BLUE)
    red_pieces = len(game_state._player_cells(PlayerColor.RED))
    blue_pieces = len(game_state._player_cells(PlayerColor.BLUE))

    # Calculate the piece difference between players
    piece_difference = red_pieces - blue_pieces

    # # Calculate the control score
    # red_control = sum(game_state.empty_neighbors(cell) for cell in red_cells)
    # blue_control = sum(game_state.empty_neighbors(cell) for cell in blue_cells)

    empty_neighbors_total = 0
    for pos, cell in game_state._state.items():
        row, col = pos
        if row < 7 and cell.player is not None:
            empty_neighbors_total += game_state.empty_neighbors(pos) 

    # Assign weights to the factors and compute the final score
    power_weight = 0.5
    piece_weight = 1.0
    control_weight = 0.2
    score = (
        power_weight * power_difference
        + piece_weight * piece_difference
        + control_weight * empty_neighbors_total
    )

    # Calculate complexity
    # achieve the Adaptive depth to Adjust the depth of your search based on the complexity of the game state
    complexity = (red_power + blue_power) * power_weight + (red_pieces + blue_pieces) + piece_weight

    return score, complexity



In [9]:
# Define a function that adjusts the depth based on the complexity

def adaptive_depth(game_state, min_depth, max_depth):
    score, complexity = evaluate_state_and_complexity(game_state)

    # You can adjust these values based on your specific requirements
    low_complexity_threshold = 10
    high_complexity_threshold = 20

    if complexity <= low_complexity_threshold:
        return max_depth
    elif complexity >= high_complexity_threshold:
        return min_depth
    else:
        # Linear interpolation between min_depth and max_depth
        depth_range = max_depth - min_depth
        complexity_range = high_complexity_threshold - low_complexity_threshold
        depth = max_depth - ((complexity - low_complexity_threshold) / complexity_range) * depth_range
        return int(depth)


In [10]:
from copy import deepcopy

def score_action(action, game_state, maximizing_player):
    new_state = deepcopy(game_state)
    new_state.apply_action(action)
    score, complexity = evaluate_state_and_complexity(game_state)

    # Invert the score if it's Blue's turn (minimizing player)
    if not maximizing_player:
        score = -score

    return score

def sorted_legal_actions(color, game_state, maximizing_player):
    legal_actions = get_legal_actions(color, game_state)
    return sorted(legal_actions, key=lambda action: score_action(action, game_state, maximizing_player), reverse=True)

In [11]:
# Transposition table是一个缓存，它存储了游戏树中先前评估的位置的结果，允许搜索算法重复使用这些结果并节省时间。为了实现换位表，你可以使用Python字典来存储不同游戏状态的评估分数。
# create a hash function to represent the game state as a unique key.

# 通过使用换位表，带有α-β修剪的最小化搜索将重新使用以前探索过的游戏状态的评估分数，从而降低时间和空间的复杂性。这种优化的有效性取决于游戏树中遇到的转置的数量。

import hashlib

def hash_game_state(game_state):
    # Convert the game_state._state into a sorted list of tuples
    board_list = sorted([(cell, state.player, state.power) for cell, state in game_state._state.items()])

    # Convert the sorted list of tuples into a string
    board_string = str(board_list)

    # Create a hash from the board_string
    return hashlib.md5(board_string.encode('utf-8')).hexdigest()

In [12]:
# def minimax_alpha_beta(game_state, depth, alpha, beta, maximizing_player):
#     if depth == 0 or is_terminal(game_state):
#         return evaluate_state(game_state)

#     legal_actions = get_legal_actions(game_state._turn_color, game_state)

#     if maximizing_player:
#         max_eval = float('-inf')
#         for action in legal_actions:
#             game_state.apply_action(action)
#             eval = minimax_alpha_beta(game_state, depth - 1, alpha, beta, False)
#             game_state.undo_action()
#             max_eval = max(max_eval, eval)
#             alpha = max(alpha, eval)
#             if beta <= alpha:
#                 break
#         return max_eval

#     else:
#         min_eval = float('inf')
#         for action in legal_actions:
#             game_state.apply_action(action)
#             eval = minimax_alpha_beta(game_state, depth - 1, alpha, beta, True)
#             game_state.undo_action()
#             min_eval = min(min_eval, eval)
#             beta = min(beta, eval)
#             if beta <= alpha:
#                 break
#         return min_eval

import time

def minimax_alpha_beta(game_state, depth, alpha, beta, maximizing_player, transposition_table, start_time, time_limit):
    # Check if the time limit has been exceeded
    if time.time() - start_time > time_limit:
        return None

    if depth == 0 or is_terminal(game_state):
        score, complexity = evaluate_state_and_complexity(game_state)
        return score

    game_state_hash = hash_game_state(game_state)

    # Check if the game state is already in the transposition table
    if game_state_hash in transposition_table:
        return transposition_table[game_state_hash]

    legal_actions = sorted_legal_actions(game_state._turn_color, game_state, maximizing_player)

    if maximizing_player:
        max_eval = float('-inf')
        for action in legal_actions:
            game_state.apply_action(action)
            eval = minimax_alpha_beta(game_state, depth - 1, alpha, beta, False, transposition_table, start_time, time_limit)
            game_state.undo_action()
            if eval is None:
                return None
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        transposition_table[game_state_hash] = max_eval
        return max_eval

    else:
        min_eval = float('inf')
        for action in legal_actions:
            game_state.apply_action(action)
            eval = minimax_alpha_beta(game_state, depth - 1, alpha, beta, True, transposition_table, start_time, time_limit)
            game_state.undo_action()
            if eval is None:
                return None
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        transposition_table[game_state_hash] = min_eval
        return min_eval



4. Determine the best action: Call the minimax function for the current game state and depth, and store the evaluation values for all legal actions. The best action is the one with the maximum evaluation value for the Red agent and the minimum evaluation value for the Blue agent.

In [13]:
from referee.game.board import Board, IllegalActionException


# def find_best_action(color, board, depth, maximizing_player):
#     best_eval = float('-inf') if maximizing_player else float('inf')
#     best_action = None

#     for action in get_legal_actions(color, board):
#         new_state = deepcopy(board)
#         try:
#             new_state.apply_action(action)
#             if new_state._total_power <= 49:
#                 eval = minimax_alpha_beta(new_state, depth - 1, float('-inf'), float('inf'), not maximizing_player)
#                 if maximizing_player:
#                     if eval > best_eval:
#                         best_eval = eval
#                         best_action = action
#                 else:
#                     if eval < best_eval:
#                         best_eval = eval
#                         best_action = action
#         except IllegalActionException:
#             pass

#     return best_action
def find_best_action(color, board, max_depth, time_limit, maximizing_player):
    best_eval = float('-inf') if maximizing_player else float('inf')
    best_action = None
    start_time = time.time()

    transposition_table = {}


    for depth in range(1, max_depth + 1):
        temp_best_eval = best_eval
        temp_best_action = best_action

        for action in get_legal_actions(color, board):
            new_state = deepcopy(board)
            try:
                new_state.apply_action(action)
                if new_state._total_power <= 49:
                    eval = minimax_alpha_beta(new_state, depth - 1, float('-inf'), float('inf'), not maximizing_player, transposition_table, start_time, time_limit)
                    
                    if eval is None:  # Time limit exceeded
                        break


                    if maximizing_player:
                        if eval > temp_best_eval:
                            temp_best_eval = eval
                            temp_best_action = action
                    else:
                        if eval < temp_best_eval:
                            temp_best_eval = eval
                            temp_best_action = action
            except IllegalActionException:
                pass

        # If the time limit is exceeded, break out of the outer loop as well
        if eval is None:
            break

        # Update best_eval and best_action if a better action was found
        if maximizing_player and temp_best_eval > best_eval:
            best_eval = temp_best_eval
            best_action = temp_best_action
        elif not maximizing_player and temp_best_eval < best_eval:
            best_eval = temp_best_eval
            best_action = temp_best_action

        # Check if the allotted time has passed, break out of the loop if it has
        current_time = time.time()
        if current_time - start_time > time_limit:
            break

    return best_action



5. Apply the best action: Update the game state by applying the best action found in the previous step for each agent.

In [14]:
from agent import Agent
from referee.game import PlayerColor

agentA = Agent(PlayerColor.BLUE)
# Create an instance of your game state (assuming you have a Board class)
board = my_board  # Replace this with the actual way to create an instance of your game state
print(my_board.render())
# Set the search depth and maximizing player
search_depth = 3
maximizing_player = True  # True for Red, False for Blue

adaptive_search_depth = adaptive_depth(board, 3, 5)

# Call the find_best_action function
# best_action = find_best_action(board.turn_color, board, search_depth, maximizing_player)

best_action = find_best_action(board.turn_color, board, max_depth=5, time_limit=2, maximizing_player=True)
# Print the best action
turnA = agentA.turn(PlayerColor.BLUE, best_action)


Testing: I am playing as blue
                         ..     
                     ..      ..     
                 ..      ..      ..     
             ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
     ..      ..      ..      ..      ..      ..     
 ..      r1      ..      ..      ..      ..      ..     
     ..      ..      ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
             ..      ..      ..      ..     
                 ..      ..      ..     
                     ..      ..     
                         ..     

Testing: BLUE SPAWN at 5-1


In [1]:
!python3 -m referee agent agent

*******************************************************************************
Welcome to Infexion referee version 2023.0.1.

Conduct a game of Infexion between 2 Agent classes.

Run `python -m referee --help` for additional usage information.
*******************************************************************************
* referee : all messages printed by referee/wrapper modules begin with *
* referee : (any other lines of output must be from your Agent class).
* referee : 
* referee : wrapping player 1 [agent:Agent] as RED...
* referee : wrapping player 2 [agent:Agent] as BLUE...
* referee : let the game begin!
* referee : player RED is initialising
Testing: I am playing as red
* referee : player BLUE is initialising
Testing: I am playing as blue
* referee : RED to play (turn 1) ...
* referee : RED plays action ACK
* referee ! player error: ILLEGAL ACTION: Unknown action ACK
* referee : game over, winner is BLUE
* referee @ result: player 2 [agent:Agent]


In [2]:
# COMP30024 Artificial Intelligence, Semester 1 2023
# Project Part B: Game Playing Agent

from referee.game import PlayerColor, Action, SpawnAction, SpreadAction, HexPos, HexDir, Board, IllegalActionException, \
    BOARD_N
from typing import Optional


# This is the entry point for your game playing agent. Currently, the agent
# simply spawns a token at the centre of the board if playing as RED, and
# spreads a token at the centre of the board if playing as BLUE. This is
# intended to serve as an example of how to use the referee API -- obviously
# this is not a valid strategy for actually playing the game!

class Agent:
    def __init__(self, color: PlayerColor, **referee: dict):
        """
        Initialise the agent.
        """
        self._color = color
        self.board = Board()  # Create a new board instance
        self.board.render()

        match color:
            case PlayerColor.RED:
                print("Red Typhon is gonna destroy you!! ๐˙Ⱉ˙๐ ")
            case PlayerColor.BLUE:
                print("Blue Typhon is gonna destroy you!! (〃'▽'〃)")

    def action(self, **referee: dict) -> Action:
        """
        Return the next action to take.
        """
        # match self._color:
        #     case PlayerColor.RED:
        #         return SpawnAction(HexPos(3, 3))
        #     case PlayerColor.BLUE:
        #         # This is going to be invalid... BLUE never spawned!
        #         return SpreadAction(HexPos(3, 3), HexDir.Up)
        return minimax_decision(self.board)

    def turn(self, color: PlayerColor, action: Action, **referee: dict):
        """
        Update the agent with the last player's action.
        """
        try:
            self.board.apply_action(action)
        except IllegalActionException as e:
            print(f"Error: Illegal action '{action}' from player {color}: {e}")

        print(referee["time_remaining"])
        # Your previous code for printing actions
        match action:
            case SpawnAction(cell):
                print(f"Testing: {color} SPAWN at {cell}")
            case SpreadAction(cell, direction):
                print(f"Testing: {color} SPREAD from {cell}, {direction}")


def extract_positions(from_board: Board, color: Optional[PlayerColor] = None) -> [HexPos]:
    """
    Extract empty or colored cells from the board
    """
    return [hex_pos for hex_pos, cell_state in from_board._state.items()
            if color is None and cell_state.player is None or
            cell_state.player == color]


def filter_cells(from_board):
    team_color = from_board._turn_color
    opponent_color = team_color.opponent

    team_cells = []
    opponent_cells = []
    empty_cells = []

    team_power = 0
    opponent_power = 0

    for hex_pos, cell_state in from_board._state.items():
        if cell_state.player is team_color:
            team_cells.append(hex_pos)
            team_power += cell_state.power
        elif cell_state.player is opponent_color:
            opponent_cells.append(hex_pos)
            opponent_power += cell_state.power
        else:
            empty_cells.append(hex_pos)

    return team_cells, team_power, opponent_cells, opponent_power, empty_cells


def get_neighbour_cells(from_board: Board, from_cells: [HexPos]):
    """
    Get all cells covered by the spread action
    """
    to_cells = [
        from_cell + hex_dir * (i + 1)
        for from_cell in from_cells
        for hex_dir in HexDir
        for i in range(from_board[from_cell].power)
    ]
    return list(set(to_cells))


def get_spread_actions(from_board: Board) -> [SpreadAction]:
    """
    Assign spread action in all direction to a cell and store them in a list
    """
    color = from_board._turn_color

    colored_positions = extract_positions(from_board, color)

    spread_actions = [SpreadAction(position, hex_dir)
                      for position in colored_positions
                      for hex_dir in HexDir]

    return spread_actions


def get_spawn_actions(from_board: Board):
    """
    Filter all empty cells on a board and match them to the spawn action
    """

    color = from_board._turn_color

    empty_positions = extract_positions(from_board)

    spawn_actions = [SpawnAction(position)
                     for position in empty_positions]

    if len(from_board._history) < 24:
        opponent_positions = extract_positions(from_board, color.opponent)
        neighbour_cells = get_neighbour_cells(from_board, opponent_positions)

        spawn_actions = [x for x in spawn_actions if x.cell not in neighbour_cells]

    return spawn_actions


def get_legal_actions(from_board: Board):
    """
    Return all legal actions.
    If total power exceed 49, return do not include spawn actions anymore
    """
    spawn_actions = get_spawn_actions(from_board)
    spread_actions = get_spread_actions(from_board)

    if from_board._total_power >= 49:
        return spread_actions
    return spawn_actions + spread_actions


def dominant(from_board: Board):
    win_power_r = 0
    win_piece_r = 0
    win_power_q = 0
    win_piece_q = 0

    color = from_board._turn_color
    opponent_color = color.opponent

    for r in range(BOARD_N):
        team_power_r = 0
        team_power_q = 0
        team_piece_r = 0
        team_piece_q = 0

        opponent_power_r = 0
        opponent_power_q = 0
        opponent_piece_r = 0
        opponent_piece_q = 0
        for q in range(BOARD_N):
            current_cell = from_board._state[HexPos(q, r)]
            if current_cell.player == color:
                team_power_q += current_cell.power
                team_piece_q += 1
            elif current_cell.player == opponent_color:
                opponent_power_q += current_cell.power
                opponent_piece_q += 1

            current_cell = from_board._state[HexPos(r, q)]
            if current_cell.player == color:
                team_power_r += current_cell.power
                team_piece_r += 1
            elif current_cell.player == opponent_color:
                opponent_power_r += current_cell.power
                opponent_piece_r += 1

        if team_power_r > opponent_power_r:
            win_power_r += 1
        if team_piece_r > opponent_piece_r:
            win_piece_r += 1
        if team_power_q > opponent_power_q:
            win_power_q += 1
        if team_piece_q > opponent_piece_q:
            win_piece_q += 1

    return win_power_r, win_power_q, win_piece_r, win_piece_q


def evaluate_power(from_board: Board):
    color = from_board._turn_color
    opponent_color = color.opponent

    team_power = from_board._color_power(color)
    opponent_power = from_board._color_power(opponent_color)

    return team_power, team_power - opponent_power


def evaluate_piece(from_board: Board):
    team_tokens = extract_positions(from_board, from_board._turn_color)
    opponent_tokens = extract_positions(from_board, from_board._turn_color.opponent)

    team_tokens_num = len(team_tokens)
    opponent_token_num = len(opponent_tokens)

    piece_diff = team_tokens_num - opponent_token_num

    vulnerable_tokens = [x for x in team_tokens if x in get_neighbour_cells(from_board, opponent_tokens)]
    vulnerable_tokens_num = len(vulnerable_tokens)

    aggressive_tokens = [y for y in opponent_tokens if y in get_neighbour_cells(from_board, team_tokens)]
    aggressive_tokens_num = len(aggressive_tokens)

    attack_benefit = aggressive_tokens_num - vulnerable_tokens_num

    # risk_signal = False
    # for token in team_tokens:
    #     if board[token].power == 6:
    #         risk_signal = True

    return team_tokens_num, piece_diff, vulnerable_tokens_num, aggressive_tokens_num, attack_benefit


def evaluate(from_board: Board, pre_power_diff):
    # if from_board.game_over:
    #     return 9999

    # win_power_r, win_power_q, win_piece_r, win_piece_q = dominant(from_board)

    team_cells, team_power, opponent_cells, opponent_power, empty_cells = filter_cells(from_board)

    piece_diff = len(team_cells) - len(opponent_cells)
    power_diff = team_power - opponent_power

    # total_power, power_diff = evaluate_power(from_board)
    # total_piece, piece_diff, vulnerable_tokens_num, aggressive_tokens_num, attack_benefit = evaluate_piece(from_board)
    #
    # # piece_gain = total_piece - pre_piece
    # # power_gain = total_power - pre_power
    # net_power_diff = abs(power_diff - pre_power_diff)
    #
    # win_piece_weight = 1
    # win_power_weight = 1
    # piece_gain_weight = 1
    # power_gain_weight = 0.5
    # power_diff_weight = 1
    # piece_diff_weight = 1
    # attack_benefit_weight = 3
    # net_power_diff_weight = 5
    #
    # # if len(board._history) > 30:
    # #     net_power_diff_weight = 4
    #
    # score = win_power_weight * (win_power_r + win_power_q) + \
    #         win_piece_weight * (win_piece_r + win_piece_q)
    # power_diff_weight * power_diff + \
    # piece_diff_weight * piece_diff
    # net_power_diff_weight * net_power_diff
    # attack_benefit_weight * attack_benefit + \
    # piece_gain_weight * piece_gain + \
    # power_gain_weight * power_gain + \

    return piece_diff + power_diff


def adaptive_depth(from_board):
    turn = len(from_board._history)

    return 2


def minimax_decision(from_board: Board):
    max_value = float('-inf')
    best_action = None
    depth = 2
    is_maximizing = True
    alpha = float('-inf')
    beta = float('inf')

    for action in get_legal_actions(from_board):
        from_board.apply_action(action)
        value = minimax_value(from_board, depth, alpha, beta, not is_maximizing, 0)
        from_board.undo_action()

        if value > max_value:
            max_value = value
            best_action = action

    return best_action


def minimax_value(from_board: Board, depth: int, alpha, beta, is_maximizing: bool, current_power_diff):
    if depth == 0 or from_board.game_over:
        return evaluate(from_board, current_power_diff)

    legal_actions = get_legal_actions(from_board)

    if is_maximizing:
        max_value = float('-inf')

        for action in legal_actions:
            pre_power_diff = key_factor(from_board)

            from_board.apply_action(action)
            value = minimax_value(from_board, depth - 1, alpha, beta, not is_maximizing, pre_power_diff)
            from_board.undo_action()

            if value > max_value:
                max_value = value

            alpha = max(alpha, max_value)
            if beta <= alpha:
                break

        return max_value

    else:
        min_value = float('inf')

        for action in legal_actions:
            pre_power_diff = key_factor(from_board)

            from_board.apply_action(action)
            value = minimax_value(from_board, depth - 1, alpha, beta, not is_maximizing, pre_power_diff)
            from_board.undo_action()

            if value < min_value:
                min_value = value

            beta = min(beta, min_value)
            if beta <= alpha:
                break

        return min_value


def key_factor(from_board: Board):
    team_power = from_board._color_power(from_board._turn_color)
    opponent_power = from_board._color_power(from_board._turn_color.opponent)

    power_diff = team_power - opponent_power

    return power_diff

SyntaxError: invalid character '」' (U+300D) (1168922825.py, line 277)